#Ben Tisinger 
2.17.2025 
Jane Street Coding Challenge

In [ ]:
test = print('Hello World')
test

In [40]:
import requests
import os
from pyspark.sql import SparkSession
import tempfile

def download_and_read_five_parquets(urls):
    """
    Downloads and reads exactly 5 parquet files from Google Drive URLs.
    Returns DataFrames named DF1 through DF5.
    """
    # Validate exact number of URLs
    if len(urls) != 1:
        raise ValueError("Exactly 5 URLs must be provided")
    
    # Initialize SparkSession outside the function
    global spark
    
    if 'spark' not in globals():
        spark = SparkSession.builder.appName("FiveParquetReader").getOrCreate()
        
    dataframes = {}
    temp_dirs = []  # Keep track of temp directories
    
    for i, url in enumerate(urls):
        file_id = url.split('/d/')[1].split('/')[0]
        download_url = f"https://drive.google.com/uc?id={file_id}&export=download"
        temp_dir = tempfile.mkdtemp(prefix='gdrive_temp_')
        temp_dirs.append(temp_dir)  # Store for cleanup
        
        file_path = os.path.join(temp_dir, f"temp_parquet_file_{i}.parquet")
        try:
            print(f"Downloading file {i+1}...")
            response = requests.get(download_url, stream=True)
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                        
            # Cache the DataFrame to ensure it's materialized
            df = spark.read.parquet(file_path).cache()
            
            # Name DataFrames DF1 through DF5
            df_name = f"DF{i+1}"
            dataframes[df_name] = df
            
            print(f"\nDataset Schema for {df_name}:")
            df.printSchema()
            print(f"\nFirst few rows for {df_name}:")
            df.show()
            
        except Exception as e:
            raise Exception(f"Error processing file {i+1}: {str(e)}")
    
    return {
        'dataframes': dataframes,
        'temp_dirs': temp_dirs
    }

# Example usage
urls = [
    "https://drive.google.com/file/d/13RMTXranhe3HkQ6rmwEV5nBoN4iuOgGa/view?usp=sharing",
    #"https://drive.google.com/file/d/1NeT7C2ro3M1NVQ2kg52IzjLRLCV6pGN3/view?usp=sharing",
    #"https://drive.google.com/file/d/1example2/view?usp=sharing",
    #"https://drive.google.com/file/d/1example3/view?usp=sharing",
    #"https://drive.google.com/file/d/1example4/view?usp=sharing"
]

result = download_and_read_five_parquets(urls)
dfs = result['dataframes']

# Access DataFrames DF1 through DF5
DF1 = dfs.get("DF1")
#DF2 = dfs.get("DF2")
#DF3 = dfs.get("DF3")
#DF4 = dfs.get("DF4")
#DF5 = dfs.get("DF5")

# Use any DataFrame (example)
DF1.show()




Dataset Schema for DF1:
root
 |-- date_id: short (nullable = true)
 |-- time_id: short (nullable = true)
 |-- symbol_id: byte (nullable = true)
 |-- responder_0_lag_1: float (nullable = true)
 |-- responder_1_lag_1: float (nullable = true)
 |-- responder_2_lag_1: float (nullable = true)
 |-- responder_3_lag_1: float (nullable = true)
 |-- responder_4_lag_1: float (nullable = true)
 |-- responder_5_lag_1: float (nullable = true)
 |-- responder_6_lag_1: float (nullable = true)
 |-- responder_7_lag_1: float (nullable = true)
 |-- responder_8_lag_1: float (nullable = true)


First few rows for DF1:
+-------+-------+---------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|date_id|time_id|symbol_id|responder_0_lag_1|responder_1_lag_1|responder_2_lag_1|responder_3_lag_1|responder_4_lag_1|responder_5_lag_1|responder_6_lag_1|responder_7_lag_1|responder_8_lag_1|
+-------+-------+

In [41]:
DF1.show()


+-------+-------+---------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|date_id|time_id|symbol_id|responder_0_lag_1|responder_1_lag_1|responder_2_lag_1|responder_3_lag_1|responder_4_lag_1|responder_5_lag_1|responder_6_lag_1|responder_7_lag_1|responder_8_lag_1|
+-------+-------+---------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|      0|      0|        0|      -0.44221526|      -0.32240674|        0.1435942|      -0.92689013|       -0.7822362|        -0.036595|       -1.3057463|       -0.7956767|      -0.14372352|
|      0|      0|        1|       -0.6518288|       -1.7078404|      -0.89394206|       -1.0654883|       -1.8713378|      -0.61565197|       -1.1628007|       -1.2059237|       -1.2459345|
|      0|      0|        2|       -0.6563728|     

In [44]:
# Create DataFrame from sample submission CSV
DF_SS = spark.read.csv('responders.csv', 
                       header=True, 
                       inferSchema=True)

# Display first 20 rows
print("Sample Submission Dataset Preview:")
print("-" * 80)
print("\nSchema:")
DF_SS.printSchema()
print("\nFirst 20 Rows:")
DF_SS.show(20)

Sample Submission Dataset Preview:
--------------------------------------------------------------------------------

Schema:
root
 |-- responder: string (nullable = true)
 |-- tag_0: boolean (nullable = true)
 |-- tag_1: boolean (nullable = true)
 |-- tag_2: boolean (nullable = true)
 |-- tag_3: boolean (nullable = true)
 |-- tag_4: boolean (nullable = true)


First 20 Rows:
+-----------+-----+-----+-----+-----+-----+
|  responder|tag_0|tag_1|tag_2|tag_3|tag_4|
+-----------+-----+-----+-----+-----+-----+
|responder_0| true|false| true|false|false|
|responder_1| true|false|false| true|false|
|responder_2| true| true|false|false|false|
|responder_3|false|false| true|false| true|
|responder_4|false|false|false| true| true|
|responder_5|false| true|false|false| true|
|responder_6|false|false| true|false|false|
|responder_7|false|false|false| true|false|
|responder_8|false| true|false|false|false|
+-----------+-----+-----+-----+-----+-----+

